In [ ]:
%cd /
from google.colab import drive
drive.mount('/content/gdrive/')

!ln -s /content/gdrive/My\ Drive/ ./mydrive

%cd /content/gdrive/MyDrive/projekt_inzynierski

/
Mounted at /content/gdrive/
/content/gdrive/MyDrive/projekt_inzynierski


In [ ]:
!pip install sentence-transformers -q

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import torch

In [ ]:
df = pd.read_csv('df_kaggle_enriched_cleaned.csv')
df.sample(5)

,track_id,name,artist,spotify_preview_url,tags,genre,year,duration_ms,danceability,energy,...,album_name,popularity,spotify_url,explicit,album_images,spotify_id,n_tempo,n_loudness,tags_list,tags_count
31385,TRRVYAR128F930BBCD,Non-Alignment Pact,Pere Ubu,https://p.scdn.co/mp3-preview/b6c9de17352c3c9c...,"experimental, post_punk, avant_garde",NaN,1977,200600,0.214,0.816,...,The Modern Dance,31.0,https://open.spotify.com/track/0rSECa18k7Gaok4...,False,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",0rSECa18k7Gaok4eizJvuw,0.653425,0.817699,"['experimental', 'post_punk', 'avant_garde']",3
10685,TRDJQMZ128EF3569F7,The Last Goodbye,The Kills,https://p.scdn.co/mp3-preview/28935db17a582488...,"alternative, indie, classic_rock, indie_rock, ...",Metal,2012,219682,0.515,0.281,...,Blood Pressures,43.0,https://open.spotify.com/track/7D1ONerlSPQOpq1...,False,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",7D1ONerlSPQOpq1EfV84uU,0.411273,0.819506,"['alternative', 'indie', 'classic_rock', 'indi...",8
35847,TRXWEZE128F423E3AE,El loco,Babasónicos,https://p.scdn.co/mp3-preview/ad132b59d7fd67da...,"rock, psychedelic_rock",NaN,2001,185826,0.687,0.387,...,Jessico,70.0,https://open.spotify.com/track/2jEjxOyKqMpkcTh...,False,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",2jEjxOyKqMpkcThWl697hf,0.471847,0.788127,"['rock', 'psychedelic_rock']",2
26018,TRJAVOL128F4284C3B,Ceremony,The Appleseed Cast,https://p.scdn.co/mp3-preview/1ffabc6f33463dde...,"indie, instrumental, post_rock",New Age,2006,260000,0.453,0.842,...,Peregrine,9.0,https://open.spotify.com/track/7mmOxJnjQIiVT9n...,False,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",7mmOxJnjQIiVT9nZaMxQ6w,0.490420,0.816772,"['indie', 'instrumental', 'post_rock']",3
34384,TRUQHDN128F4227D15,Misery (acoustic),Maroon 5,https://p.scdn.co/mp3-preview/11e86c27f5581eea...,"rock, acoustic, male_vocalists",NaN,2010,226306,0.729,0.599,...,Hands All Over (Revised International Deluxe),47.0,https://open.spotify.com/track/7ieV1TBSMyf5sFi...,False,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",7ieV1TBSMyf5sFiPh9X2ai,0.406078,0.823717,"['rock', 'acoustic', 'male_vocalists']",3


In [ ]:
df['tags'] = df['tags'].str.replace('_', ' ')
df['tags'] = df['tags'].fillna("")
df.sample(5)

,track_id,name,artist,spotify_preview_url,tags,genre,year,duration_ms,danceability,energy,...,album_name,popularity,spotify_url,explicit,album_images,spotify_id,n_tempo,n_loudness,tags_list,tags_count
19504,TROODKL128F428673B,Shogun,Trivium,https://p.scdn.co/mp3-preview/7187dc7254b8843e...,"metal, heavy metal, thrash metal, metalcore, m...",NaN,2008,714986,0.227,0.9640,...,Shogun (Special Edition),41.0,https://open.spotify.com/track/5FaEX9EwutRqVvO...,False,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",5FaEX9EwutRqVvOZsJ27G3,0.585860,0.874753,"['metal', 'heavy_metal', 'thrash_metal', 'meta...",5
45403,TRWSSOB128F4295632,Ming Tran,Gob,https://p.scdn.co/mp3-preview/d557dc63999085e2...,"punk, punk rock, ska",NaN,2002,154186,0.334,0.9550,...,Foot In Mouth Disease,18.0,https://open.spotify.com/track/0UNbpyz1cqowPWh...,True,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",0UNbpyz1cqowPWhgyleWNm,0.759735,0.882420,"['punk', 'punk_rock', 'ska']",3
9672,TRUEJZL12903CE94AF,I Can't Get Started,Jamie Cullum,https://p.scdn.co/mp3-preview/796f46adff7f840f...,"jazz, piano",NaN,2011,311144,0.544,0.0899,...,Pointless Nostalgic (Remastered),29.0,https://open.spotify.com/track/1koaigmrZEI4T0p...,False,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",1koaigmrZEI4T0pu4PfM1P,0.497892,0.685884,"['jazz', 'piano']",2
36311,TRGMWGN128F92F1DB8,Summer In Paris (Starring Anggun),DJ Cam,https://p.scdn.co/mp3-preview/0d6f21a8533fc1d4...,"chillout, downtempo, french",Electronic,2015,360400,0.611,0.5700,...,Soulshine,48.0,https://open.spotify.com/track/349344mLV84YaND...,False,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",349344mLV84YaND0fNHAXa,0.539869,0.780695,"['chillout', 'downtempo', 'french']",3
5899,TRCRVZT128F932C081,Let's Get Lost,Elliott Smith,https://p.scdn.co/mp3-preview/79bedc8659094ce3...,"indie, folk, indie rock, singer songwriter, ac...",NaN,2005,145360,0.526,0.3110,...,From A Basement On The Hill,33.0,https://open.spotify.com/track/2hHJhyR9zGSuk20...,False,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",2hHJhyR9zGSuk204Y5AcMC,0.385588,0.680415,"['indie', 'folk', 'indie_rock', 'singer_songwr...",9


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model = SentenceTransformer('intfloat/multilingual-e5-base', device=device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:
required_cols = ['track_id', 'name', 'artist', 'tags']

In [ ]:
tags = df['tags']
tags

,tags
0,"rock, alternative, indie, alternative rock, in..."
1,"rock, alternative, indie, pop, alternative roc..."
2,"rock, alternative, alternative rock, 90s, grunge"
3,"rock, alternative, indie, alternative rock, in..."
4,"rock, alternative, indie, alternative rock, in..."
...,...
49761,
49762,"rock, alternative rock, japanese, cover"
49763,"metal, metalcore, post hardcore"
49764,


In [ ]:
embeddings = model.encode(
    tags.tolist(),
    batch_size=64,
    convert_to_numpy=True,
    normalize_embeddings=True,
    show_progress_bar=True
)

Batches:   0%|          | 0/778 [00:00<?, ?it/s]

In [ ]:
embeddings

array([[-0.00501017,  0.01898571, -0.02007603, ..., -0.01049391,
        -0.05588296,  0.0657074 ],
       [ 0.00328209,  0.02081618, -0.02525968, ..., -0.02236444,
        -0.04646278,  0.04495326],
       [-0.00029811,  0.01794667, -0.01753925, ..., -0.0019554 ,
        -0.04001404,  0.06583691],
       ...,
       [ 0.03838515,  0.04050802, -0.00097008, ..., -0.02710878,
        -0.04771475,  0.03838398],
       [ 0.00160186,  0.05012948,  0.00090271, ..., -0.04161721,
        -0.03978097,  0.04399836],
       [-0.0068757 ,  0.03136002, -0.0167717 , ..., -0.02534982,
        -0.04039215,  0.04295585]], dtype=float32)

In [ ]:
df_embeddings = df[['track_id', 'name', 'artist', 'tags']].copy()
df_embeddings['tags_embedding'] = embeddings.tolist()
df_embeddings

,track_id,name,artist,tags,tags_embedding
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,"rock, alternative, indie, alternative rock, in...","[-0.005010171327739954, 0.01898571476340294, -..."
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,"rock, alternative, indie, pop, alternative roc...","[0.003282089252024889, 0.02081618271768093, -0..."
2,TROUVHL128F426C441,Come as You Are,Nirvana,"rock, alternative, alternative rock, 90s, grunge","[-0.000298114464385435, 0.017946673557162285, ..."
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,"rock, alternative, indie, alternative rock, in...","[0.0032091771718114614, 0.018290618434548378, ..."
4,TRLNZBD128F935E4D8,Creep,Radiohead,"rock, alternative, indie, alternative rock, in...","[0.007342000957578421, 0.020590929314494133, -..."
...,...,...,...,...,...
49761,TRQYCFV128F9322F50,Ryusei Rocket,アンティック-珈琲店-,,"[0.0016018569003790617, 0.050129484385252, 0.0..."
49762,TRHQCSH128F42724B7,Colors Of The Wind,ACIDMAN,"rock, alternative rock, japanese, cover","[-0.014852749183773994, 0.014049864374101162, ..."
49763,TRZRODK128F92D68D7,The Revelation,coldrain,"metal, metalcore, post hardcore","[0.03838515281677246, 0.04050801694393158, -0...."
49764,TRGLMEM128F9322F63,Koi no Dependence,アンティック-珈琲店-,,"[0.0016018569003790617, 0.050129484385252, 0.0..."


In [ ]:
import numpy as np

In [ ]:
np.save('./tag_embeddings.npy', embeddings)

In [ ]:
df_embeddings.to_parquet('./df_embeddings.parquet', index=False)

In [ ]:
df['tags_list'] = df['tags'].apply(lambda x: [tag.strip().lower() for tag in x.split(',')])

In [ ]:
all_tags = [tag for tags in df['tags_list'] for tag in tags]
unique_tags = list(set(all_tags))

unique_tags = [tag for tag in unique_tags if tag]

In [ ]:
tag_texts = [f"passage: {t}" for t in unique_tags]

In [ ]:
tag_embeddings = model.encode(
    tag_texts,
    batch_size=64,
    convert_to_numpy=True,
    normalize_embeddings=True,
    show_progress_bar=True
).astype("float32")

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
df_tag_embeddings = pd.DataFrame({
    'tag': unique_tags,
    'tag_embedding': tag_embeddings.tolist()
})

In [ ]:
np.save('./unique_tag_embeddings.npy', tag_embeddings)
df_tag_embeddings.to_parquet('./df_unique_tag_embeddings.parquet', index=False)